# WISONet δD Retrievals
This notebook demonstrates how averaging kernels and a priori values affect satellite δD retrievals.


In [ ]:
# Sample a priori and AK matrix
import numpy as np
import matplotlib.pyplot as plt

# Simulated state vector (true)
true_dd = np.linspace(-300, 0, 20)

# A priori: climatology
apriori = np.full(20, -150)

# Averaging kernel: diagonal dominant
AK = np.diag(np.linspace(0.1, 0.9, 20))

# Retrieved state
retrieved = apriori + AK @ (true_dd - apriori)

# Plot
plt.plot(true_dd, np.arange(20), label='True')
plt.plot(retrieved, np.arange(20), label='Retrieved')
plt.plot(apriori, np.arange(20), '--', label='A priori')
plt.gca().invert_yaxis()
plt.xlabel('δD (‰)')
plt.ylabel('Pressure Level Index')
plt.legend()
plt.title('Effect of Averaging Kernel on δD Retrieval')
plt.grid(True)
plt.show()

In [13]:
!curl -L "https://rice.box.com/shared/static/bcoy3ob0dme3umpurqmf0p6o48bznkj1" -o SWING3_2006_subset.nc
!curl -L "https://rice.box.com/shared/static/bcoy3ob0dme3umpurqmf0p6o48bznkj1" -o SWING3_2006_subset.nc


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100     8    0     8    0     0      7      0 --:--:--  0:00:01 --:--:--     7
100 12.7M  100 12.7M    0     0  4172k      0  0:00:03  0:00:03 --:--:-- 7277k


In [11]:
!ls -lh SWING3_2006_subset.nc


-rw-r--r-- 1 root root 22K Jun 12 20:12 SWING3_2006_subset.nc


In [14]:
import xarray as xr

ds = xr.open_dataset("SWING3_2006_subset.nc")
ds


<xarray.Dataset> Size: 13MB
Dimensions:  (lon: 128, lat: 64, lev: 17, month: 12)
Coordinates:
  * lon      (lon) float64 1kB 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
  * lat      (lat) float64 512B 87.86 85.1 82.31 79.53 ... -82.31 -85.1 -87.86
  * month    (month) int32 48B 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: lev
Data variables:
    p        (lev) float64 136B ...
    dD       (month, lev, lat, lon) float32 7MB ...
    q        (month, lev, lat, lon) float32 7MB ...